# Baseline Greedy Packing with Rotation Tightening

This notebook implements:
1. Greedy placement algorithm with weighted angle generation
2. Rotation tightening (fix_direction) to minimize bounding box
3. Proper scoring and validation

In [1]:
import math
import random
from decimal import Decimal, getcontext
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """
    Generates a random angle with a distribution weighted by abs(sin(2*angle)).
    This helps place more trees in corners, and makes the packing less round.
    """
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle


def initialize_trees(num_trees, existing_trees=None, num_attempts=10):
    """
    Greedy placement algorithm: place trees one by one, moving from far away
    toward center until collision, then back up.
    """
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = list(existing_trees)

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:  # Only place the first tree at origin if starting from scratch
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            # This loop tries multiple random starting attempts and keeps the best one
            for _ in range(num_attempts):
                # The new tree starts at a position 20 from the center, at a random vector angle.
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                # Move towards center along the vector in steps of 0.5 until collision
                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    # Looking for nearby objects
                    possible_indices = tree_index.query(candidate_poly)
                    # This is the collision detection step
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                # back up in steps of 0.05 until it no longer has a collision.
                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    # No collision found even at the center. Place it at the center.
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    all_polygons = [t.polygon for t in placed_trees]
    bounds = unary_union(all_polygons).bounds

    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor

    width = maxx - minx
    height = maxy - miny
    side_length = max(width, height)

    return placed_trees, side_length

print('Greedy placement functions defined')

Greedy placement functions defined


In [4]:
# Rotation tightening functions

def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])


def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)

    hull_points = points_np[ConvexHull(points_np).vertices]

    initial_side = calculate_bbox_side_at_angle(0, hull_points)

    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun

    improvement = initial_side - found_side
    EPSILON = 1e-8

    if improvement > EPSILON:
        best_angle_deg = found_angle_deg
        best_side = Decimal(str(found_side))
    else:
        best_angle_deg = 0.0
        best_side = Decimal(str(initial_side))

    return best_side, best_angle_deg


def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]

    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0]) / float(scale_factor)

    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])

    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center

    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees


def fix_direction(trees):
    """Apply rotation tightening to minimize bounding box."""
    if len(trees) <= 1:
        return trees
    
    best_side, best_angle_deg = optimize_rotation(trees)
    if abs(best_angle_deg) > 1e-9:
        return apply_rotation(trees, best_angle_deg)
    return [t.clone() for t in trees]

print('Rotation tightening functions defined')

Rotation tightening functions defined


In [5]:
# Scoring and validation functions

def get_side_length(trees):
    """Calculate the side length of the bounding square."""
    if not trees:
        return Decimal('0')
    
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    
    width = maxx - minx
    height = maxy - miny
    return max(width, height)


def get_score(trees, n):
    """Calculate score contribution for n trees: s^2 / n"""
    side = get_side_length(trees)
    return float(side ** 2) / n


def has_overlap(trees):
    """Check if any two trees overlap."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print('Scoring and validation functions defined')

Scoring and validation functions defined


In [6]:
# Run the greedy placement algorithm for all configurations
random.seed(42)  # For reproducibility

start_time = time.time()

all_configurations = {}  # n -> list of trees
all_side_lengths = {}  # n -> side length

current_placed_trees = []

print('Starting greedy placement...')
for n in range(1, 201):
    # Build n-tree configuration from (n-1)-tree configuration
    current_placed_trees, side = initialize_trees(n, existing_trees=current_placed_trees, num_attempts=10)
    
    # Apply rotation tightening
    optimized_trees = fix_direction(current_placed_trees)
    optimized_side = get_side_length(optimized_trees)
    
    # Keep the better result
    if optimized_side < side:
        all_configurations[n] = optimized_trees
        all_side_lengths[n] = optimized_side
        # Update current_placed_trees for next iteration
        current_placed_trees = [t.clone() for t in optimized_trees]
    else:
        all_configurations[n] = [t.clone() for t in current_placed_trees]
        all_side_lengths[n] = side
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f'n={n:3d}: side={float(all_side_lengths[n]):.6f}, elapsed={elapsed:.1f}s')

elapsed = time.time() - start_time
print(f'\nTotal time: {elapsed:.1f}s')

Starting greedy placement...


n= 20: side=4.070858, elapsed=0.4s


n= 40: side=5.788087, elapsed=1.0s


n= 60: side=6.962340, elapsed=1.7s


n= 80: side=8.147965, elapsed=2.6s


...

n=120: side=9.881660, elapsed=5.0s


n=140: side=10.279545, elapsed=6.5s


n=160: side=11.374610, elapsed=8.1s


n=180: side=12.046099, elapsed=10.0s


n=200: side=12.589317, elapsed=12.0s

Total time: 12.0s


In [7]:
# Calculate total score
total_score = 0.0
for n in range(1, 201):
    score_n = float(all_side_lengths[n] ** 2) / n
    total_score += score_n

print(f'Total score: {total_score:.6f}')
print(f'Target score to beat: 68.954209')

Total score: 163.194562
Target score to beat: 68.954209


In [8]:
# Validate all configurations for overlaps
print('Validating configurations for overlaps...')
overlap_count = 0
for n in range(1, 201):
    if has_overlap(all_configurations[n]):
        print(f'  OVERLAP at n={n}')
        overlap_count += 1

if overlap_count == 0:
    print('All configurations are valid (no overlaps)')
else:
    print(f'Found {overlap_count} configurations with overlaps')

Validating configurations for overlaps...


All configurations are valid (no overlaps)


In [9]:
# Create submission dataframe
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

tree_data = []
for n in range(1, 201):
    for tree in all_configurations[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

# Round to 6 decimal places
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Prepend 's' to values
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(f'Submission shape: {submission.shape}')
print(submission.head(10))

Submission shape: (20100, 3)
                x           y          deg
id                                        
001_0        s0.0        s0.0  s230.193647
002_0    s0.02595   s0.003889  s236.959515
002_1  s-0.300048   s0.131267    s15.76974
003_0    s0.20321  s-0.120374  s270.655246
003_1  s-0.138686  s-0.195254    s49.46547
003_2   s0.535893   s0.284504  s324.261909
004_0   s0.222728  s-0.070546  s279.195434
004_1  s-0.104258  s-0.195368   s58.005658
004_2   s0.491596   s0.379247  s332.802097
004_3  s-0.641443  s-0.363834  s145.144147


In [10]:
# Save submission
submission.to_csv('/home/submission/submission.csv')
print('Submission saved to /home/submission/submission.csv')

# Also save to experiment folder
submission.to_csv('/home/code/experiments/001_baseline_greedy/submission.csv')
print('Submission also saved to experiment folder')

Submission saved to /home/submission/submission.csv
Submission also saved to experiment folder


In [11]:
# Final summary
print('='*60)
print('BASELINE GREEDY EXPERIMENT SUMMARY')
print('='*60)
print(f'Total Score: {total_score:.6f}')
print(f'Target Score: 68.954209')
print(f'Overlap Count: {overlap_count}')
print(f'Time Elapsed: {elapsed:.1f}s')
print('='*60)

BASELINE GREEDY EXPERIMENT SUMMARY
Total Score: 163.194562
Target Score: 68.954209
Overlap Count: 0
Time Elapsed: 12.0s


In [12]:
# Let's check the sample submission score for comparison
sample_df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Sample submission shape: {sample_df.shape}')
print(sample_df.head())

# Parse and score the sample submission
def parse_and_score_submission(df):
    """Parse submission and calculate total score."""
    total_score = 0.0
    
    for n in range(1, 201):
        # Get rows for this configuration
        prefix = f'{n:03d}_'
        group_data = df[df['id'].str.startswith(prefix)]
        
        if len(group_data) == 0:
            continue
            
        trees = []
        for _, row in group_data.iterrows():
            x = str(row['x'])[1:]  # Remove 's' prefix
            y = str(row['y'])[1:]
            deg = str(row['deg'])[1:]
            trees.append(ChristmasTree(x, y, deg))
        
        side = get_side_length(trees)
        score_n = float(side ** 2) / n
        total_score += score_n
    
    return total_score

sample_score = parse_and_score_submission(sample_df)
print(f'Sample submission score: {sample_score:.6f}')

Sample submission shape: (20100, 4)
      id          x           y    deg
0  001_0       s0.0        s0.0  s90.0
1  002_0       s0.0        s0.0  s90.0
2  002_1  s0.202736  s-0.511271  s90.0
3  003_0       s0.0        s0.0  s90.0
4  003_1  s0.202736  s-0.511271  s90.0


Sample submission score: 173.652299


In [14]:
# Debug rotation tightening - let's check if it's actually improving anything
# Test on a specific configuration

test_n = 50
test_trees = all_configurations[test_n]
original_side = get_side_length(test_trees)

# Try rotation tightening
best_side, best_angle = optimize_rotation(test_trees)
print(f'n={test_n}: original_side={float(original_side):.6f}, optimized_side={float(best_side):.6f}, angle={best_angle:.2f}')
print(f'Improvement: {float(original_side - best_side):.6f}')

n=50: original_side=6.471459, optimized_side=6.471459, angle=0.00
Improvement: 0.000000


In [15]:
# Analyze score contributions by n
print('Score contributions by n:')
cumulative = 0.0
for n in range(1, 201):
    score_n = float(all_side_lengths[n] ** 2) / n
    cumulative += score_n
    if n <= 20 or n % 20 == 0:
        print(f'n={n:3d}: side={float(all_side_lengths[n]):.4f}, score_n={score_n:.4f}, cumulative={cumulative:.4f}')

Score contributions by n:
n=  1: side=0.8386, score_n=0.7033, cumulative=0.7033
n=  2: side=1.3335, score_n=0.8890, cumulative=1.5924
n=  3: side=1.7499, score_n=1.0207, cumulative=2.6130
n=  4: side=2.1111, score_n=1.1142, cumulative=3.7272
n=  5: side=2.1111, score_n=0.8913, cumulative=4.6186
n=  6: side=2.1111, score_n=0.7428, cumulative=5.3614
n=  7: side=2.1782, score_n=0.6778, cumulative=6.0392
n=  8: side=2.2159, score_n=0.6138, cumulative=6.6529
n=  9: side=2.5798, score_n=0.7395, cumulative=7.3924
n= 10: side=2.8551, score_n=0.8152, cumulative=8.2076
n= 11: side=3.0219, score_n=0.8302, cumulative=9.0378
n= 12: side=3.1012, score_n=0.8014, cumulative=9.8393
n= 13: side=3.3053, score_n=0.8404, cumulative=10.6796
n= 14: side=3.4700, score_n=0.8601, cumulative=11.5397
n= 15: side=3.5245, score_n=0.8282, cumulative=12.3679
n= 16: side=3.5130, score_n=0.7713, cumulative=13.1392
n= 17: side=3.5778, score_n=0.7530, cumulative=13.8922
n= 18: side=3.8500, score_n=0.8235, cumulative=14.7